In [ ]:
import os
import shutil
from google.colab import files


# # Upload your Kaggle API key (kaggle.json)
files.upload()

kaggle_path = '/root/.config/kaggle'
os.makedirs(kaggle_path, exist_ok=True)
shutil.move('kaggle.json', os.path.join(kaggle_path,'kaggle.json'))

os.chmod(os.path.join(kaggle_path,'kaggle.json'), 600)
print("kaggle.json has been moved and configured.")



In [ ]:
from kaggle.api.kaggle_api_extended import KaggleApi

# Set up the Kaggle API client
api = KaggleApi()

api.authenticate()

dataset = "litvinenko630/insurance-claims"
download_dir = 'Insurance claims data.csv'

os.makedirs(download_dir,exist_ok=True)
api.dataset_download_files(dataset, path=download_dir, unzip=True)

print("Dataset Downloaded and extracted")

In [ ]:
import pandas as pd


data = pd.read_csv('Insurance claims data.csv')
data.head()

In [ ]:
#Preprocessing
data.info()

In [ ]:
data_copy = data.copy()


#Dropping Policy ID column
columns_to_drop = ['policy_id']
data_copy.drop(columns=columns_to_drop, inplace=True)

data_copy.head()

In [ ]:
print(data_copy['claim_status'].unique())
print(data_copy['is_esc'].unique())

In [ ]:
#changing the binary columns to Boolean

binary_columns = [col for col in data_copy.columns if data_copy[col].nunique() == 2 and data_copy[col].dtype == 'object']
print(binary_columns)


for col in binary_columns:
  data_copy[col] = data_copy[col].map({'Yes':True, 'No':False})


data_copy.info()

In [ ]:
#Extracting numeric Values

data_copy['max_torque'] = data_copy['max_torque'].str.extract(r'(\d+\.\d+|\d+)').astype(float)
data_copy['max_power'] = data_copy['max_power'].str.extract(r'(\d+\.\d+|\d+)').astype(float)

data_copy.head()

In [ ]:
data['length_to_width_ratio'] = data['length'] / data['width']
data_copy.head()

In [ ]:
continous_col = ['subscription_length','turning_radius']
for col in continous_col:
  data_copy[col] = (data_copy[col] - data_copy[col].mean()) / data_copy[col].std()

data_copy.head(10)

In [ ]:
categorical_columns = ['region_code','fuel_type','segment','model','engine_type','transmission_type','steering_type']
data_copy = pd.get_dummies(data_copy, columns=categorical_columns, drop_first=True)


data_copy.head()

In [ ]:
data_copy.info()

data_copy.columns = data_copy.columns.str.replace(r'[.\s]', '_', regex=True)


# Mapping dataset headers to SQL table headers
column_mapping = {
    'engine_type_1_2_L_K_Series_Engine': 'engine_type_1_2_l_k_series_engine',
    'engine_type_1_2_L_K12N_Dualjet': 'engine_type_1_2_l_k12n_dualjet',
    'engine_type_1_5_L_U2_CRDi': 'engine_type_1_5_l_u2_crdi',
    'engine_type_1_5_Turbocharged_Revotorq': 'engine_type_1_5_turbocharged_revotorq',
    'engine_type_1_5_Turbocharged_Revotron': 'engine_type_1_5_turbocharged_revotron',
    'engine_type_F8D_Petrol_Engine': 'engine_type_f8d_petrol_engine',
    'engine_type_G12B': 'engine_type_g12b',
    'engine_type_K_Series_Dual_jet': 'engine_type_k_series_dual_jet',
    'engine_type_K10C': 'engine_type_k10c',
    'engine_type_i-DTEC': 'engine_type_i_dtec'
}

# Rename dataset columns
data_copy = data_copy.rename(columns=column_mapping)

# Verify the renamed columns
print(data_copy.columns)



In [17]:
cleansed_file_path = 'Cleansed_insurance_claims_data (1).csv'
data_copy.to_csv(cleansed_file_path, index=False)

**SQL**

In [ ]:
pip install psycopg2

In [ ]:
print(data_copy['is_esc'].unique())


print(data_copy.columns)

In [ ]:
import psycopg2
import pandas as pd

conn = psycopg2.connect(
    dbname = 'InsuranceWarehouse',
    user = 'postgres',
    password = 'your password',
    host = 'localhost',
    port ='5432'
)

cursor = conn.cursor()


customers = data_copy[['customer_age','region_density']].drop_duplicates()

for _, row in customers.iterrows():
    cursor.execute(
        "INSERT INTO Customers (customer_age,region_density) VALUES (%s, %s)",
        (int(row['customer_age']),int(row['region_density']))
    )

vehicles = data_copy[['vehicle_age','fuel_type_Diesel','fuel_type_Petrol','airbags','displacement','max_torque','max_power']].drop_duplicates()

for _, row in vehicles.iterrows():
    cursor.execute(
        "INSERT INTO Vehicles (vehicle_age, fuel_type, airbags, displacement, max_torque, max_power) VALUES ( %s, %s, %s, %s, %s, %s)",
        (row['vehicle_age'], 'Diesel' if row['fuel_type_Diesel'] else 'Petrol', row['airbags'],row['displacement'],row['max_torque'],row['max_power'])
    )

features = data_copy[['is_esc', 'is_adjustable_steering', 'is_tpms', 'is_parking_sensors', 'is_parking_camera', 'is_front_fog_lights', 'is_rear_window_wiper', 'is_rear_window_washer', 'is_rear_window_defogger', 'is_brake_assist', 'is_power_door_locks', 'is_central_locking', 'is_power_steering', 'is_driver_seat_height_adjustable', 'is_day_night_rear_view_mirror', 'is_ecw', 'is_speed_alert']].drop_duplicates()
for _, row in features.iterrows():
    cursor.execute(
        """
        INSERT INTO Features (is_esc, is_adjustable_steering, is_tpms, is_parking_sensors, is_parking_camera, is_front_fog_lights, is_rear_window_wiper, is_rear_window_washer, is_rear_window_defogger, is_brake_assist, is_power_door_locks, is_central_locking, is_power_steering, is_driver_seat_height_adjustable, is_day_night_rear_view_mirror, is_ecw, is_speed_alert)
        VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
        """,
        tuple(row)
    )

regions = data_copy[['region_code_C10', 'region_code_C11', 'region_code_C12', 'region_code_C13', 'region_code_C14', 'region_code_C15', 'region_code_C16', 'region_code_C17', 'region_code_C18', 'region_code_C19', 'region_code_C2', 'region_code_C20', 'region_code_C21', 'region_code_C22', 'region_code_C3', 'region_code_C4', 'region_code_C5', 'region_code_C6', 'region_code_C7', 'region_code_C8', 'region_code_C9']].drop_duplicates()
for _, row in regions.iterrows():
    cursor.execute(
        """
        INSERT INTO Regions (region_code_c10, region_code_c11, region_code_c12, region_code_c13, region_code_c14, region_code_c15, region_code_c16, region_code_c17, region_code_c18, region_code_c19, region_code_c2, region_code_c20, region_code_c21, region_code_c22, region_code_c3, region_code_c4, region_code_c5, region_code_c6, region_code_c7, region_code_c8, region_code_c9)
        VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
        """,
        tuple(row)
    )

models = data_copy[['model_M2','model_M3','model_M4','model_M5','model_M6','model_M7','model_M8','model_M9','model_M10','model_M11']]

for _,row in models.iterrows():
    cursor.execute(
        """
            INSERT INTO Models (model_m2, model_m3, model_m4, model_m5, model_m6, model_m7, model_m8, model_m9, model_m10, model_m11) VALUES
            (%s,%s,%s,%s,%s,%s,%s,%s,%s,%s)
        """,
        tuple(row)
    )

engines = data_copy[[
    'engine_type_1_2_l_k_series_engine', 
    'engine_type_1_2_l_k12n_dualjet', 
    'engine_type_1_5_l_u2_crdi', 
    'engine_type_1_5_turbocharged_revotorq',
    'engine_type_1_5_turbocharged_revotron',
    'engine_type_f8d_petrol_engine',
    'engine_type_g12b', 
    'engine_type_k_series_dual_jet',
    'engine_type_k10c',
    'engine_type_i_dtec'
]].drop_duplicates()

# Insert data into the Engines table
for _, row in engines.iterrows():
    cursor.execute(
        """
        INSERT INTO Engines (
            engine_type_1_2_l_k_series_engine,
            engine_type_1_2_l_k12n_dualjet,
            engine_type_1_5_l_u2_crdi,
            engine_type_1_5_turbocharged_revotorq,
            engine_type_1_5_turbocharged_revotron,
            engine_type_f8d_petrol_engine,
            engine_type_g12b,
            engine_type_k_series_dual_jet,
            engine_type_k10c,
            engine_type_i_dtec
        )
        VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
        """,
        tuple(row)
    )

def get_dimension_id(field, table, columns, values):
    query = f"SELECT {field}_id FROM {table} WHERE " + " AND ".join([f"{col}= %s" for col in columns]) 
    cursor.execute(query,values)
    result = cursor.fetchone()
    if result:
        return result[0]
    else:
        return None
    

for _,row in data_copy.iterrows():
    customer_id = get_dimension_id("customer","Customers",["customer_age","region_density"],[row['customer_age'],row['region_density']])
    vehicle_id = get_dimension_id('vehicle',"Vehicles", ["vehicle_age", "fuel_type", "airbags", "displacement"], [row["vehicle_age"], 'Diesel' if row['fuel_type_Diesel'] else 'Petrol', row["airbags"], row["displacement"]])
    feature_id = get_dimension_id('feature',"Features",[
        "is_esc", "is_adjustable_steering", "is_tpms", 
        "is_parking_sensors", "is_parking_camera", "is_front_fog_lights",
        "is_rear_window_wiper", "is_rear_window_washer", "is_rear_window_defogger", 
        "is_brake_assist", "is_power_door_locks", "is_central_locking", 
        "is_power_steering", "is_driver_seat_height_adjustable", 
        "is_day_night_rear_view_mirror", "is_ecw", "is_speed_alert"
    ],
    [
        row["is_esc"], row["is_adjustable_steering"], row["is_tpms"], 
        row["is_parking_sensors"], row["is_parking_camera"], row["is_front_fog_lights"],
        row["is_rear_window_wiper"], row["is_rear_window_washer"], row["is_rear_window_defogger"], 
        row["is_brake_assist"], row["is_power_door_locks"], row["is_central_locking"], 
        row["is_power_steering"], row["is_driver_seat_height_adjustable"], 
        row["is_day_night_rear_view_mirror"], row["is_ecw"], row["is_speed_alert"]
    ]
    )
    region_id = get_dimension_id('region',"Regions",[
        "region_code_C10", "region_code_C11", "region_code_C12", 
        "region_code_C13", "region_code_C14", "region_code_C15", 
        "region_code_C16", "region_code_C17", "region_code_C18", 
        "region_code_C19", "region_code_C2", "region_code_C20", 
        "region_code_C21", "region_code_C22", "region_code_C3", 
        "region_code_C4", "region_code_C5", "region_code_C6", 
        "region_code_C7", "region_code_C8", "region_code_C9"
    ],
    [
        row["region_code_C10"], row["region_code_C11"], row["region_code_C12"], 
        row["region_code_C13"], row["region_code_C14"], row["region_code_C15"], 
        row["region_code_C16"], row["region_code_C17"], row["region_code_C18"], 
        row["region_code_C19"], row["region_code_C2"], row["region_code_C20"], 
        row["region_code_C21"], row["region_code_C22"], row["region_code_C3"], 
        row["region_code_C4"], row["region_code_C5"], row["region_code_C6"], 
        row["region_code_C7"], row["region_code_C8"], row["region_code_C9"]
    ]
    )
    engine_id = get_dimension_id('engine',"Engines", [
        "engine_type_1_2_l_k_series_engine", 
        "engine_type_1_2_l_k12n_dualjet", 
        "engine_type_1_5_l_u2_crdi", 
        "engine_type_1_5_turbocharged_revotorq",
        "engine_type_1_5_turbocharged_revotron",
        "engine_type_f8d_petrol_engine",
        "engine_type_g12b", 
        "engine_type_k_series_dual_jet",
        "engine_type_k10c",
        "engine_type_i_dtec"
    ],
    [
        row["engine_type_1_2_l_k_series_engine"], 
        row["engine_type_1_2_l_k12n_dualjet"], 
        row["engine_type_1_5_l_u2_crdi"], 
        row["engine_type_1_5_turbocharged_revotorq"],
        row["engine_type_1_5_turbocharged_revotron"],
        row["engine_type_f8d_petrol_engine"],
        row["engine_type_g12b"], 
        row["engine_type_k_series_dual_jet"],
        row["engine_type_k10c"],
        row["engine_type_i_dtec"]
    ]
    )
    model_id = get_dimension_id(
    'model',
    "Models",
    [
        "model_M10", "model_M11", "model_M2", 
        "model_M3", "model_M4", "model_M5", 
        "model_M6", "model_M7", "model_M8", "model_M9"
    ],
    [
        row["model_M10"], row["model_M11"], row["model_M2"], 
        row["model_M3"], row["model_M4"], row["model_M5"], 
        row["model_M6"], row["model_M7"], row["model_M8"], row["model_M9"]
    ]
    )

    if None in [customer_id, vehicle_id, feature_id, region_id, engine_id, model_id]:
        continue

    cursor.execute(
        """
        INSERT INTO Claims (subscription_length, claim_status, customer_id, vehicle_id, feature_id, region_id, engine_id, model_id)
        VALUES (%s, %s, %s, %s, %s, %s, %s, %s)
        """,
        (row["subscription_length"], row["claim_status"], customer_id, vehicle_id, feature_id, region_id, engine_id, model_id)
    )


# Commit changes and close the connection
conn.commit()
cursor.close()
conn.close()

print("Engines table has been successfully populated.")